## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/kaggle_data1.zip"

Archive:  /content/drive/MyDrive/kaggle_data1.zip
  inflating: kaggle_data/track_0.csv  
  inflating: kaggle_data/track_1.csv  
  inflating: kaggle_data/track_10.csv  
  inflating: kaggle_data/track_100.csv  
  inflating: kaggle_data/track_101.csv  
  inflating: kaggle_data/track_102.csv  
  inflating: kaggle_data/track_103.csv  
  inflating: kaggle_data/track_104.csv  
  inflating: kaggle_data/track_105.csv  
  inflating: kaggle_data/track_106.csv  
  inflating: kaggle_data/track_107.csv  
  inflating: kaggle_data/track_108.csv  
  inflating: kaggle_data/track_109.csv  
  inflating: kaggle_data/track_11.csv  
  inflating: kaggle_data/track_110.csv  
  inflating: kaggle_data/track_111.csv  
  inflating: kaggle_data/track_112.csv  
  inflating: kaggle_data/track_113.csv  
  inflating: kaggle_data/track_114.csv  
  inflating: kaggle_data/track_115.csv  
  inflating: kaggle_data/track_116.csv  
  inflating: kaggle_data/track_117.csv  
  inflating: kaggle_data/track_118.csv  
  inflating: 

In [ ]:
import os
import pandas as pd
import numpy as np
path = r'/content/kaggle_data'  
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                print(dir_entry_path.split("\\")[-1])
                pass

/content/kaggle_data/track_0.csv
/content/kaggle_data/track_1.csv
/content/kaggle_data/track_10.csv
/content/kaggle_data/track_100.csv
/content/kaggle_data/track_101.csv
/content/kaggle_data/track_102.csv
/content/kaggle_data/track_103.csv
/content/kaggle_data/track_104.csv
/content/kaggle_data/track_105.csv
/content/kaggle_data/track_106.csv
/content/kaggle_data/track_107.csv
/content/kaggle_data/track_108.csv
/content/kaggle_data/track_109.csv
/content/kaggle_data/track_11.csv
/content/kaggle_data/track_110.csv
/content/kaggle_data/track_111.csv
/content/kaggle_data/track_112.csv
/content/kaggle_data/track_113.csv
/content/kaggle_data/track_114.csv
/content/kaggle_data/track_115.csv
/content/kaggle_data/track_116.csv
/content/kaggle_data/track_117.csv
/content/kaggle_data/track_118.csv
/content/kaggle_data/track_119.csv
/content/kaggle_data/track_12.csv
/content/kaggle_data/track_120.csv
/content/kaggle_data/track_121.csv
/content/kaggle_data/track_122.csv
/content/kaggle_data/track_

Переведём типы движений в числа

In [ ]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [ ]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [ ]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    if a == -1:
        st = 1250
    else:
        st = a
            
    if b == -1:
        end = -1250
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1

    print(name)
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return -1

# рисуем графики ускорений
#     if cut:
#         ax1.plot(df.iloc[st:end]['gFx'],c='g')
#         ax1.plot(df.iloc[st:end]['gFy'],c='r')
#         ax1.plot(df.iloc[st:end]['gFz'],c='b')
#     else:
#         ax1.plot(df['gFx'],c='g')
#         ax1.plot(df['gFy'],c='r')
#         ax1.plot(df['gFz'],c='b')
        
# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    v = int(Acc)
    amplitude = Acc.max() - Acc.min()
    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
    print()
    
    return amplitude

Ampl = np.zeros(len(Data)) 

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[2].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Ampl[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib
/content/kaggle_data/track_0.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


TypeError: ignored

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [ ]:
def get_prediction(ampl):
    if ampl < 1:
        return 0 #стояние
    if ampl > :
        return 5
    if ampl >  5 :
        return 2 #бег
    return 1 #ходьба

In [ ]:
len(Data), len(Ampl), len(Target)

(146, 146, 146)

In [ ]:
type(Data)

list

In [ ]:
Ampl_new = []

i=0
for elem in Ampl:
    if not (math.isnan(elem) or elem == -1):
        i+=1
        Ampl_new.append(elem)
    else:
        Ampl_new.append(2)
len(Ampl_new), i

(146, 140)

Создадим таблицу, содержащую номер трека и предсказание.

In [ ]:
prediction = [get_prediction(a) for a in Ampl]

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,1
1,1,1
2,2,1
3,3,2
4,4,1
5,5,1
6,6,1
7,7,5
8,8,0
9,9,0


In [ ]:
Res.to_csv("first_submit.csv", index=False)